Setup


In [2]:
# === Cell 0: Imports ===
import os, re, glob
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import matplotlib.pyplot as plt
from numpy.fft import rfft, rfftfreq

print("Ready!")


Ready!


1. Analysis Parameters


In [3]:
# === Cell 1: Tunable parameters ===
SR        = 48000            # analysis sample rate
F0        = 1000.0           # test tone (Hz)
NFFT_MAIN = 65536            # FFT size (tone/silence spectra)
BAND_HZ   = 5.0              # ±Hz around f0/harmonics to integrate power
TRIM_SEC  = 1.0              # cut head+tail (avoid transients)
MAX_H     = 10               # up to 10th harmonic

# Short-time SNR (per-frame)
ST_FRAME  = int(0.050*SR)    # 50 ms
ST_HOP    = int(0.025*SR)    # 25 ms
NFFT_ST   = 16384

DATA_ROOT = Path("data")     # Step-1 output folder
PLOTS_DIR = Path("plots_step2"); PLOTS_DIR.mkdir(parents=True, exist_ok=True)


2. Helper Functions


In [5]:
# === Cell 2: Helpers (load, FFT, metrics, scoring) ===
def load_wav_center(path, sr=SR, trim_sec=TRIM_SEC):
    """Mono, resample to SR, remove DC, keep steady middle (trim head/tail)."""
    y, srx = sf.read(path, dtype='float32')
    if y.ndim > 1: y = y.mean(axis=1)
    if srx != sr:
        y = librosa.resample(y, orig_sr=srx, target_sr=sr)
    if len(y) > 2*int(trim_sec*sr):
        y = y[int(trim_sec*sr):-int(trim_sec*sr)]
    y = y - np.mean(y)
    return y, sr

def mag_spectrum(y, sr=SR, nfft=NFFT_MAIN, use_hann=True):
    """Single-sided magnitude spectrum (Hann window + amplitude normalization)."""
    y = y.astype(np.float64)
    if use_hann:
        w = np.hanning(len(y)); y = y * w
        norm = (np.sum(w)/2.0)
    else:
        norm = len(y)/2.0
    if len(y) < nfft:
        y = np.pad(y, (0, nfft-len(y)))
    Y = rfft(y[:nfft])
    mag = np.abs(Y) / (norm if norm>0 else 1.0)
    freqs = rfftfreq(nfft, 1/sr)
    return freqs, mag

def band_power(freqs, mag, f_center, width_hz=BAND_HZ):
    sel = (freqs >= f_center-width_hz) & (freqs <= f_center+width_hz)
    return float(np.sum(mag[sel]**2))

def analyze_tone(tone_path):
    """Return dict with f0, SNR(dB), THD(%), THD+N(dB) + spectra & time."""
    y, sr = load_wav_center(tone_path, SR)
    freqs, mag = mag_spectrum(y, sr)
    sel = (freqs >= F0-BAND_HZ) & (freqs <= F0+BAND_HZ)
    if not np.any(sel): return None
    f0_est = freqs[sel][np.argmax(mag[sel])]
    P1 = band_power(freqs, mag, f0_est)
    Ph = 0.0
    for h in range(2, MAX_H+1):
        fc = h*f0_est
        if fc >= freqs[-1]: break
        Ph += band_power(freqs, mag, fc)
    Ptot = float(np.sum(mag**2))
    Pn = max(Ptot - P1 - Ph, 1e-20)
    SNR_dB  = 10*np.log10(P1/Pn)
    THD_pct = 100*np.sqrt(Ph/P1) if P1>0 else np.nan
    THDN_dB = 10*np.log10((Ph+Pn)/P1) if P1>0 else np.nan
    return dict(f0=f0_est, SNR_dB=SNR_dB, THD_pct=THD_pct, THDN_dB=THDN_dB,
                freqs=freqs, mag=mag, y=y)

def analyze_silence(sil_path):
    """Return dict with noise_floor_dbfs + spectra & time."""
    y, sr = load_wav_center(sil_path, SR)
    freqs, mag = mag_spectrum(y, sr)
    noise_floor_dbfs = 20*np.log10(np.median(mag)+1e-20)
    return dict(noise_floor_dbfs=noise_floor_dbfs, freqs=freqs, mag=mag, y=y)

# scoring: map raw metrics to 0–100 and combine
def _lin(x, x0, x1):
    if x <= x0: return 0.0
    if x >= x1: return 100.0
    return 100.0*(x-x0)/(x1-x0)

def score_metrics(SNR_dB, THD_pct, THDN_dB, noise_floor_dbfs, weights=(1,1.2,1.2,1)):
    snr_s = _lin(SNR_dB,      40, 70)     # higher better
    thd_s = _lin(-THD_pct,    -2, -0.1)   # lower % better
    thn_s = _lin(-THDN_dB,    30, 60)     # more negative better
    nff_s = _lin(-noise_floor_dbfs, 55, 80)
    w = np.array(weights, float); w /= w.sum()
    final = w.dot([snr_s, thd_s, thn_s, nff_s])
    return snr_s, thd_s, thn_s, nff_s, final
